In [1]:
# importing required libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn import preprocessing, metrics
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
sns.set(style="white", color_codes=True)
import warnings
warnings.filterwarnings("ignore")

In [30]:
# (1)Principal Component Analysis
dst_CC = pd.read_csv('CC GENERAL.csv')
dst_CC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8950 entries, 0 to 8949
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   CUST_ID                           8950 non-null   object 
 1   BALANCE                           8950 non-null   float64
 2   BALANCE_FREQUENCY                 8950 non-null   float64
 3   PURCHASES                         8950 non-null   float64
 4   ONEOFF_PURCHASES                  8950 non-null   float64
 5   INSTALLMENTS_PURCHASES            8950 non-null   float64
 6   CASH_ADVANCE                      8950 non-null   float64
 7   PURCHASES_FREQUENCY               8950 non-null   float64
 8   ONEOFF_PURCHASES_FREQUENCY        8950 non-null   float64
 9   PURCHASES_INSTALLMENTS_FREQUENCY  8950 non-null   float64
 10  CASH_ADVANCE_FREQUENCY            8950 non-null   float64
 11  CASH_ADVANCE_TRX                  8950 non-null   int64  
 12  PURCHA

In [4]:
dst_CC.head()

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
3,C10004,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,NaN,0.000000,12
4,C10005,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12


In [31]:
dst_CC.isnull().any()

CUST_ID                             False
BALANCE                             False
BALANCE_FREQUENCY                   False
PURCHASES                           False
ONEOFF_PURCHASES                    False
INSTALLMENTS_PURCHASES              False
CASH_ADVANCE                        False
PURCHASES_FREQUENCY                 False
ONEOFF_PURCHASES_FREQUENCY          False
PURCHASES_INSTALLMENTS_FREQUENCY    False
CASH_ADVANCE_FREQUENCY              False
CASH_ADVANCE_TRX                    False
PURCHASES_TRX                       False
CREDIT_LIMIT                         True
PAYMENTS                            False
MINIMUM_PAYMENTS                     True
PRC_FULL_PAYMENT                    False
TENURE                              False
dtype: bool

In [32]:
dst_CC.fillna(dst_CC.mean(), inplace=True)
dst_CC.isnull().any()

CUST_ID                             False
BALANCE                             False
BALANCE_FREQUENCY                   False
PURCHASES                           False
ONEOFF_PURCHASES                    False
INSTALLMENTS_PURCHASES              False
CASH_ADVANCE                        False
PURCHASES_FREQUENCY                 False
ONEOFF_PURCHASES_FREQUENCY          False
PURCHASES_INSTALLMENTS_FREQUENCY    False
CASH_ADVANCE_FREQUENCY              False
CASH_ADVANCE_TRX                    False
PURCHASES_TRX                       False
CREDIT_LIMIT                        False
PAYMENTS                            False
MINIMUM_PAYMENTS                    False
PRC_FULL_PAYMENT                    False
TENURE                              False
dtype: bool

In [7]:
a = dst_CC.iloc[:,1:-1]
b = dst_CC.iloc[:,-1]
print(a.shape,b.shape)

(8950, 16) (8950,)


In [34]:
#(1a) Apply PCA on CC Dataset 
pca = PCA(3)
a_pca = pca.fit_transform(a)
prplDf = pd.DataFrame(data = a_pca, columns = ['principal cmpnt 1', 'principal cmpnt 2', 'principal cmpnt 3'])
finalDf = pd.concat([prplDf, dst_CC.iloc[:,-1]], axis = 1)
finalDf.head()

,principal cmpnt 1,principal cmpnt 2,principal cmpnt 3,TENURE
0,-4326.383979,921.566882,183.708383,12
1,4118.916665,-2432.846346,2369.969289,12
2,1497.907641,-1997.578694,-2125.631328,12
3,1394.548536,-1488.743453,-2431.799649,12
4,-3743.351896,757.342657,512.476492,12


In [35]:
#1b) Apply K Means on PCA Result
A = finalDf.iloc[:,0:-1]
b = finalDf.iloc[:,-1]
nclusters = 3 # this is the k in kmeans
km = KMeans(n_clusters=nclusters)
km.fit(A)

# predict the cluster for each data point
b_cluster_kmeans = km.predict(A)


# Summary of the predictions made by the classifier
print(classification_report(b, b_cluster_kmeans, zero_division=1))
print(confusion_matrix(b, b_cluster_kmeans))


train_acc = accuracy_score(b, b_cluster_kmeans)
print("\nAccuracy of our training dataset with PCA:", train_acc)


#Calculate silhouette Score
scr = metrics.silhouette_score(A, b_cluster_kmeans)
print("Sihouette Score: ",scr) 

              precision    recall  f1-score   support

           0       0.00      1.00      0.00       0.0
           1       0.00      1.00      0.00       0.0
           2       0.00      1.00      0.00       0.0
           6       1.00      0.00      0.00     204.0
           7       1.00      0.00      0.00     190.0
           8       1.00      0.00      0.00     196.0
           9       1.00      0.00      0.00     175.0
          10       1.00      0.00      0.00     236.0
          11       1.00      0.00      0.00     365.0
          12       1.00      0.00      0.00    7584.0

    accuracy                           0.00    8950.0
   macro avg       0.70      0.30      0.00    8950.0
weighted avg       1.00      0.00      0.00    8950.0

[[   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [ 175    1   28    0    0    0    0    0    0    0]
 [ 173    2   15    0    0   

In [10]:
#1c) Scaling +PCA + KMeans
a = dst_CC.iloc[:,1:-1]
b = dst_CC.iloc[:,-1]
print(a.shape,b.shape)

(8950, 16) (8950,)


In [37]:
#Scaling
scaler = StandardScaler()
scaler.fit(a)
X_scaled_array = scaler.transform(a)
#PCA
pca = PCA(3)
x_pca = pca.fit_transform(X_scaled_array)
prplDf = pd.DataFrame(data = x_pca, columns = ['principal cmpt 1', 'principal cmpt 2','principal cmpt 3'])
finalDf = pd.concat([prplDf, dst_CC.iloc[:,-1]], axis = 1)
finalDf.head()

,principal cmpt 1,principal cmpt 2,principal cmpt 3,TENURE
0,-1.718893,-1.072941,0.535672,12
1,-1.169306,2.509323,0.628048,12
2,0.938414,-0.382602,0.161170,12
3,-0.907503,0.045860,1.521705,12
4,-1.637830,-0.684977,0.425652,12


In [12]:
A = finalDf.iloc[:,0:-1]
b = finalDf["TENURE"]
print(A.shape,b.shape)

(8950, 3) (8950,)


In [38]:
A_train, A_test, b_train, b_test = train_test_split(A,b, test_size=0.34,random_state=0)
nclusters = 3 
# this is the k in kmeans
km = KMeans(n_clusters=nclusters)
km.fit(A_train,b_train)


# predict the cluster for each training data point
b_clust_train = km.predict(A_train)

# Summary of the predictions made by the classifier
print(classification_report(b_train, b_clust_train, zero_division=1))
print(confusion_matrix(b_train, b_clust_train))

train_acc = accuracy_score(b_train, b_clust_train)
print("Accuracy of our training dataset with PCA:", train_acc)

#Calculate sihouette Score
scr = metrics.silhouette_score(A_train, b_clust_train)
print("Sihouette Score: ",scr) 

              precision    recall  f1-score   support

           0       0.00      1.00      0.00       0.0
           1       0.00      1.00      0.00       0.0
           2       0.00      1.00      0.00       0.0
           6       1.00      0.00      0.00     139.0
           7       1.00      0.00      0.00     135.0
           8       1.00      0.00      0.00     128.0
           9       1.00      0.00      0.00     118.0
          10       1.00      0.00      0.00     151.0
          11       1.00      0.00      0.00     262.0
          12       1.00      0.00      0.00    4974.0

    accuracy                           0.00    5907.0
   macro avg       0.70      0.30      0.00    5907.0
weighted avg       1.00      0.00      0.00    5907.0

[[   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [ 123   16    0    0    0    0    0    0    0    0]
 [ 126    9    0    0    0   

In [39]:
# predict the cluster for each testing data point
b_clust_test = km.predict(A_test)

# Summary of the predictions made by the classifier
print(classification_report(b_test, b_clust_test, zero_division=1))
print(confusion_matrix(b_test, b_clust_test))

train_acc = accuracy_score(b_test, b_clust_test)
print("\nAccuracy of our training dataset with PCA:", train_acc)

#Calculate sihouette Score
scr = metrics.silhouette_score(A_test, b_clust_test)
print("Sihouette Score: ",scr) 

              precision    recall  f1-score   support

           0       0.00      1.00      0.00       0.0
           1       0.00      1.00      0.00       0.0
           2       0.00      1.00      0.00       0.0
           6       1.00      0.00      0.00      65.0
           7       1.00      0.00      0.00      55.0
           8       1.00      0.00      0.00      68.0
           9       1.00      0.00      0.00      57.0
          10       1.00      0.00      0.00      85.0
          11       1.00      0.00      0.00     103.0
          12       1.00      0.00      0.00    2610.0

    accuracy                           0.00    3043.0
   macro avg       0.70      0.30      0.00    3043.0
weighted avg       1.00      0.00      0.00    3043.0

[[   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [  53   12    0    0    0    0    0    0    0    0]
 [  47    7    1    0    0   

In [40]:
# 2)Use pd_speech_features.csv
dst_pd = pd.read_csv('pd_speech_features.csv')
dst_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 756 entries, 0 to 755
Columns: 755 entries, id to class
dtypes: float64(749), int64(6)
memory usage: 4.4 MB


In [16]:
dst_pd.head()

,id,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,...,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,0,1,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,...,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
1,0,1,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,...,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780,1
2,0,1,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,...,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
3,1,0,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,...,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603,1
4,1,0,0.32790,0.79782,0.53028,236,235,0.008162,0.002669,0.00535,...,6.1727,5.8416,6.0805,5.7621,7.7817,11.6891,8.2103,5.0559,6.1164,1


In [41]:
dst_pd.isnull().any()

id                           False
gender                       False
PPE                          False
DFA                          False
RPDE                         False
                             ...  
tqwt_kurtosisValue_dec_33    False
tqwt_kurtosisValue_dec_34    False
tqwt_kurtosisValue_dec_35    False
tqwt_kurtosisValue_dec_36    False
class                        False
Length: 755, dtype: bool

In [18]:
X = dst_pd.drop('class',axis=1).values
y = dst_pd['class'].values

In [42]:
#a)Scaling Data
scaler = StandardScaler()
X_Scale = scaler.fit_transform(X)

In [44]:
#b) Apply PCA with k =3
pca3 = PCA(n_components=3)
principalComponents = pca3.fit_transform(X_Scale)

prplDf = pd.DataFrame(data = principalComponents, columns = ['principal cmpnt 1', 'principal cmpnt 2','Principal cmpnt 3'])

finalDf = pd.concat([prplDf, dst_pd[['class']]], axis = 1)
finalDf.head()

,principal cmpnt 1,principal cmpnt 2,Principal cmpnt 3,class
0,-0.885142,-0.037383,-1.016968,1
1,-0.886440,-0.023904,-1.076722,1
2,-0.804398,-0.351203,-1.368072,1
3,1.632921,1.483096,-0.926653,1
4,1.795405,1.005976,-0.684682,1


In [21]:
X = finalDf.drop('class',axis=1).values
y = finalDf['class'].values
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.34,random_state=0)

In [45]:
#c) Support Vector Machine(SVM)

from sklearn.svm import SVC

svmClassifier = SVC()
svmClassifier.fit(X_train, y_train)

y_pred = svmClassifier.predict(X_test)

# Summary of the predictions made by the classifier
print(classification_report(y_test, y_pred, zero_division=1))
print(confusion_matrix(y_test, y_pred))
# Accuracy score
glass_acc_svc = accuracy_score(y_pred,y_test)
print('accuracy is',glass_acc_svc )

#Calculate sihouette Score
scr= metrics.silhouette_score(X_test, y_pred)
print("Sihouette Score: ",scr) 

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        16
Iris-versicolor       1.00      0.94      0.97        18
 Iris-virginica       0.92      1.00      0.96        11

       accuracy                           0.98        45
      macro avg       0.97      0.98      0.98        45
   weighted avg       0.98      0.98      0.98        45

[[16  0  0]
 [ 0 17  1]
 [ 0  0 11]]
accuracy is 0.9777777777777777
Sihouette Score:  0.6439643595874587


In [46]:
#3) Apply Linear Discriminant Analysis (LDA) on Iris.csv dataset to reduce dimensionality of data to k=2. 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
dst_iris = pd.read_csv('Iris.csv')
dst_iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             150 non-null    int64  
 1   SepalLengthCm  150 non-null    float64
 2   SepalWidthCm   150 non-null    float64
 3   PetalLengthCm  150 non-null    float64
 4   PetalWidthCm   150 non-null    float64
 5   Species        150 non-null    object 
dtypes: float64(4), int64(1), object(1)
memory usage: 7.2+ KB


In [24]:
dst_iris.isnull().any()

Id               False
SepalLengthCm    False
SepalWidthCm     False
PetalLengthCm    False
PetalWidthCm     False
Species          False
dtype: bool

In [25]:
x = dst_iris.iloc[:,1:-1]
y = dst_iris.iloc[:,-1]
print(x.shape,y.shape)

(150, 4) (150,)


In [27]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [28]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
le = LabelEncoder()
y = le.fit_transform(y)

In [48]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components=2)
X_train = lda.fit_transform(X_train, y_train)
X_test = lda.transform(X_test)
print(X_train.shape,X_test.shape)

(105, 2) (45, 2)


#4. Briefly identify the difference between PCA and LDA

Both LDA and PCA are based on linear transformations and aim to maximize the variance of the lower dimension. PCA is an unsupervised learning algorithm while LDA is a supervised learning algorithm.That is, PCA finds indications of maximum variance regardless of class labels,  while LDA finds indications of maximum class separation
